In [1]:
import auction_data, os, analytics
hist_plot = auction_data.hist_plot
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cvxpy
sns.set_style('white')
%matplotlib inline

In [2]:
tsuchiura_data = auction_data.AuctionData('tests/reference_data/tsuchiura_data.csv')

In [3]:
deviations = [-.02, -.0001, .0, .0005]
demands = [tsuchiura_data.get_counterfactual_demand(rho) for rho in deviations]
print(demands)

[0.6938393464942137, 0.25238257317903334, 0.25017018379850237, 0.24727705922396187]


In [4]:
constraints = [analytics.MarkupConstraint(max_markup=.5),
               analytics.InformationConstraint(k=.5, sample_demands=demands)]

In [7]:
installed_solvers = cvxpy.installed_solvers()
print(installed_solvers)
solvers = ['ECOS', 'CVXOPT']
print(solvers)

['ECOS', 'ECOS_BB', 'CVXOPT', 'GLPK', 'GLPK_MI', 'SCS', 'OSQP']
['ECOS', 'CVXOPT']


In [ ]:
for solver in solvers:
    if solver not in installed_solvers:
        continue
    seed = 1
    project = True
    for num_points in [1e4]:
        print(20*'=' + '\n'
              + 'seed={}, project={}, num_points={}, solver={}'.format(seed, project, num_points, solver))
        
        try:
            min_collusion_solver = analytics.MinCollusionSolver(
                data=tsuchiura_data,
                deviations=deviations,
                tolerance=2 * 1./tsuchiura_data.df_bids.shape[0],
                metric=analytics.IsNonCompetitive,
                plausibility_constraints=constraints,
                num_points=num_points,
                seed=seed,
                project=project,
                solver_type=solver
                )
            
            print('minimum share of collusive histories: {}'.format(min_collusion_solver.solution))
            min_collusion_solver.argmin.sort_values("prob", ascending=False).head(10)
            
        except Exception as e:
            print('Solver error')
            pass

seed=1, project=True, num_points=10000.0, solver=ECOS

ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +9.782e-01  -1.390e+00  +1e+03  1e+00  7e-01  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  -4.655e-01  -7.857e-01  +2e+02  5e-01  8e-02  1e-01  3e-01  0.8521  2e-02   1  1  1 |  0  0
 2  -9.547e-02  -1.275e-01  +3e+01  8e-02  8e-03  7e-03  4e-02  0.8861  2e-02   2  1  1 |  0  0
 3  -1.513e-02  -2.084e-02  +6e+00  1e-02  1e-03  1e-03  8e-03  0.8113  2e-02   1  1  1 |  0  0
 4  -1.145e-02  -1.687e-02  +5e+00  1e-02  1e-03  1e-03  8e-03  0.1309  7e-01   2  2  2 |  0  0
 5  -6.369e-03  -9.477e-03  +3e+00  7e-03  8e-04  8e-04  5e-03  0.4979  2e-01   2  2  2 |  0  0
 6  -3.952e-04  -3.387e-03  +3e+00  7e-03  7e-04  9e-04  5e-03  0.1893  8e-01   2  2  2 |  0  0
 7  -1.165e-04  -1.531e-03  +1e+00  3e-03  3e-04  5e-04  2e-03  0.7543  3e-01   2  2  2 